<a href="https://colab.research.google.com/github/y-pred/Databases/blob/main/FAISS_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FAISS- Facebook AI Similarity Search
https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.


In [30]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.1 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

In [3]:
import pandas as pd
df = pd.read_csv('/content/Tags.csv')
df

,Text,Tag
0,I like eating healthy vegetables and fruits,Health
1,Chicken is a good source of Protien,Health
2,Would you like to dance with me,Activity
3,Weekends I play cricket for fun,Activity
4,I like blue color clothes,Fashion
5,Long boots are always classic,Fashion
6,Black suits are my all time favourite,Fashion
7,Nvidia is coming with with faster GPUs,Technology
8,People nowadays are going for Apple I phone as...,Technology
9,Electric Vehicles are rapidly funding adoption...,Technology


Text Preprocessing using Spacy

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')
stop_words = nlp.Defaults.stop_words

In [11]:
def preprocess(text):
  doc = nlp(str(text))
  preprocessed_text = []
  for token in doc:
    if token.is_punct or token.like_num or token in stop_words or token.is_space:
      continue
    preprocessed_text.append(token.lemma_.lower().strip())
  return ' '.join(preprocessed_text)

In [12]:
df['Processed Text'] = df['Text'].apply(preprocess)

In [13]:
df

,Text,Tag,Processed Text
0,I like eating healthy vegetables and fruits,Health,i like eat healthy vegetable and fruit
1,Chicken is a good source of Protien,Health,chicken be a good source of protien
2,Would you like to dance with me,Activity,would you like to dance with i
3,Weekends I play cricket for fun,Activity,weekend i play cricket for fun
4,I like blue color clothes,Fashion,i like blue color clothe
5,Long boots are always classic,Fashion,long boot be always classic
6,Black suits are my all time favourite,Fashion,black suit be my all time favourite
7,Nvidia is coming with with faster GPUs,Technology,nvidia be come with with fast gpu
8,People nowadays are going for Apple I phone as...,Technology,people nowadays be go for apple i phone as the...
9,Electric Vehicles are rapidly funding adoption...,Technology,electric vehicles be rapidly fund adoption as ...


Text Encoding using Sentence Transformer

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
df['Embedding'] = df['Processed Text'].apply(model.encode)

In [24]:
vector = model.encode(df['Processed Text'])

In [27]:
vector.shape

(10, 384)

Using FAISS to create index and store vector embeddings

In [34]:
dim = vector.shape[1]

In [35]:
import faiss
index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x784cdd4ffe10> >

In [36]:
index.add(vector)

Querring FAISS Database

In [66]:
search_query = 'I like eating cauliflower'
test_pre = preprocess(search_query)
encode_pre = model.encode(test_pre)
encode_pre.shape

(384,)

FAISS expects 2d array, so next step we are converting encode_pre to a 2D array

In [49]:
import numpy as np
svec = np.array(encode_pre).reshape(1,-1)
svec.shape

(1, 384)

In [59]:
distance,pos = index.search(svec,k=2)

In [60]:
distance

array([[0.7021954, 1.5253415]], dtype=float32)

In [61]:
pos

array([[0, 1]])

In [62]:
pos[0]

array([0, 1])

In [65]:
df.Text.iloc[pos[0]]

0    I like eating healthy vegetables and fruits
1            Chicken is a good source of Protien
Name: Text, dtype: object